In [11]:

import numpy as np
from scipy.io import loadmat
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import  classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

In [12]:
data = loadmat('/content/PaviaU.mat')['paviaU']

gt = loadmat('/content/PaviaU_gt.mat')['paviaU_gt']

df = pd.DataFrame(data.reshape(data.shape[0]*data.shape[1], -1))

df.columns = [f'band{i}' for i in range(1, df.shape[-1]+1)]

df['class'] = gt.ravel()

In [13]:
X = df[df['class']!=0].iloc[:, :-1].values

y = tf.keras.utils.to_categorical(df[df['class']!=0].iloc[:, -1].values ,
                                  num_classes= np.unique(gt).shape[0],
                                  dtype='float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, stratify = y)

print(f"Train Data: {X_train.shape}\nTest Data: {X_test.shape}")

Train Data: (29943, 103)
Test Data: (12833, 103)


In [14]:
model = Sequential(name='Pavia_University')

model.add(Input(shape = X_train[0].shape, name = 'Input_Layer'))

model.add(BatchNormalization(name = 'BatchNormalization'))

model.add(Dense(units = 128, activation= 'relu', name = 'Layer1'))
model.add(Dense(units = 128, activation= 'relu', name = 'Layer2'))

model.add(Dropout(rate = 0.2, name = 'Dropout1',))

model.add(Dense(units = 64, activation= 'relu', name = 'Layer3'))
model.add(Dense(units = 64, activation= 'relu', name = 'Layer4'))

model.add(Dropout(rate = 0.2, name = 'Dropout2'))

model.add(Dense(units = 32, activation= 'relu', name = 'Layer5'))
model.add(Dense(units = 32, activation= 'relu', name = 'Layer6'))

model.add(Dense(units = y_train.shape[1], activation= 'softmax', name = 'Output_Layer'))

model.summary()

Model: "Pavia_University"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 BatchNormalization (BatchN  (None, 103)               412       
 ormalization)                                                   
                                                                 
 Layer1 (Dense)              (None, 128)               13312     
                                                                 
 Layer2 (Dense)              (None, 128)               16512     
                                                                 
 Layer3 (Dense)              (None, 128)               16512     
                                                                 
 Layer4 (Dense)              (None, 128)               16512     
                                                                 
 Dropout1 (Dropout)          (None, 128)               0         
                                                  

In [15]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x = X_train,
          y = y_train,
          validation_data = (X_test, y_test),
          epochs = 20,
          )

Epoch 1/20
936/936 [==============================] - 11s 7ms/step - loss: 0.8305 - accuracy: 0.6899 - val_loss: 0.5701 - val_accuracy: 0.7790
Epoch 2/20
936/936 [==============================] - 7s 7ms/step - loss: 0.5867 - accuracy: 0.7762 - val_loss: 0.4656 - val_accuracy: 0.8110
Epoch 3/20
936/936 [==============================] - 7s 7ms/step - loss: 0.5049 - accuracy: 0.8103 - val_loss: 0.3915 - val_accuracy: 0.8404
Epoch 4/20
936/936 [==============================] - 8s 9ms/step - loss: 0.4733 - accuracy: 0.8165 - val_loss: 0.3351 - val_accuracy: 0.8648
Epoch 5/20
936/936 [==============================] - 7s 7ms/step - loss: 0.4359 - accuracy: 0.8282 - val_loss: 0.3326 - val_accuracy: 0.8706
Epoch 6/20
936/936 [==============================] - 7s 7ms/step - loss: 0.4050 - accuracy: 0.8389 - val_loss: 0.3076 - val_accuracy: 0.8699
Epoch 7/20
936/936 [==============================] - 7s 8ms/step - loss: 0.3802 - accuracy: 0.8490 - val_loss: 0.3041 - val_accuracy: 0.8784
Epoch

In [16]:
pred = np.argmax(model.predict(X_test), axis=1)

# Classification Report
print(classification_report(pred, np.argmax(y_test, 1),
      target_names = ['1. Asphalt', '2. Meadows', '3. Gravel', '4. Trees',
                 '5. Painted metal sheets','6. Bare Soil', '7. Bitumen',
                 '8. Self-Blocking Bricks', '9. Shadows']))

402/402 [==============================] - 2s 4ms/step
                         precision    recall  f1-score   support

             1. Asphalt       0.88      0.97      0.93      1802
             2. Meadows       1.00      0.93      0.96      5970
              3. Gravel       0.60      0.86      0.71       440
               4. Trees       0.90      0.99      0.94       839
5. Painted metal sheets       1.00      1.00      1.00       405
           6. Bare Soil       0.81      0.99      0.89      1233
             7. Bitumen       0.89      0.76      0.82       466
8. Self-Blocking Bricks       0.95      0.75      0.84      1393
             9. Shadows       1.00      1.00      1.00       285

               accuracy                           0.92     12833
              macro avg       0.89      0.92      0.90     12833
           weighted avg       0.93      0.92      0.92     12833

